In [2]:
from glob import glob
from tqdm import tqdm
import subprocess
import pandas as pd
import natsort as ns
import numpy as np
import os

In [13]:
WRK = '/workdir/users/pd378/oilPCR/bulk_process_all_OIL_for_paper/'
F_LIST = WRK+'file_lists/filelist.txt' #a list of the files used for iterating and naming
count_table = pd.DataFrame() #all the counted info gets added here

with open(F_LIST) as f:
    for line in f: 
        read = line.strip()
        
        #initialize variables to 0
        R1_count = 0        
        R2_count = 0
        merge_count = 0    #total numbr of merged reads
        filter_count = 0        #merged reads remaining after quality filter
        uniq_count=0      #total number of uniqs 
        single_count=0    #number of uniqs with a single member
        uniq_max = ''      #number of reads making up the most common uniq
        split_targ = 0
        split_ribo = 0
        unsplit = 0
        otu_count = 0     #number of OTUs after clustering
        chimera_count = 0
        target_count = 0
        target_chimera = 0


        #COUNT THE RAW READS
        try:
            with open(WRK+'/rawdata/'+read+'_R1.fastq') as R1:
                for line in R1:
                  R1_count += 1
            R1_count = R1_count/4 #1 read is 4 lines in a fasq file
        except IOError as e:
            print ("Unable to open R1 file")

        try:
            with open(WRK+'/rawdata/'+read+'_R2.fastq') as R2:
               for line in R2:
                    R2_count += 1    
            R2_count = R2_count/4

        except IOError as e:
            print ("Unable to open R2 file")            
            


        
        
        #COUNT THE MERGED READS
        try:
            with open(WRK+'/s1_merge_reads/'+read+'_merged.fq') as merged:
                for line in merged:
                  merge_count += 1
            merge_count = merge_count/4
        except IOError as e:
            print ("Unable to open merged file")
                      
               
        



        #COUNT THE FILTERED READS
        try:
             with open(WRK+'/s3_quality_filter/stats/'+read+'.txt') as file:
                for line in (file):
                    if 'Filtered reads (' in line:
                        line=line.split(' ')
                        for i in line:
                            if i.isdigit():
                                filter_count = i
        except IOError as e:
            print ("Unable to open quality filter stats file")
              
            
       

        #COUNT UNIQ READS
        uniq_seqs = 'n/a'
        uniq_max = 'n/a'
        single_count='n/a'
        try:
            with open(WRK+'/s4_filter_uniques/stats/'+read+'.txt') as file:
                for line in (file):
                    if " uniques, " in line:
                        uniq_seqs=line
                    if "Min size " in line:
                        uniq_max = line
                if not uniq_seqs=='n/a':
                    uniq_seqs = uniq_seqs.split(',')
                    uniq_count = uniq_seqs[1].rstrip(' uniques')
                    single_count = uniq_seqs[2].split(' ')[1]
                    uniq_max = uniq_max.split(',')
                    uniq_max = uniq_max[2].lstrip('max ')

        except IOError as e:
            print ("Unable to open "+read+" unique read stats file")
              
            
        
        #COUNT SPLIT READS
        try:
            with open(WRK+'/s5_split_reads/ribo_reads/'+read+'_ribo.fa') as ribo:
                for line in ribo:
                  split_ribo += 1
            split_ribo = split_ribo/2
        except IOError as e:
            print ("Unable to open split ribo file")  
            
        try:
            with open(WRK+'/s5_split_reads/target_reads/'+read+'_targ.fa') as targ:
                for line in targ:
                  split_targ += 1
            split_targ = split_targ/2
        except IOError as e:
            print ("Unable to open split ribo file")              
            
    
        summary = [[read, R1_count, R2_count, merge_count, filter_count, uniq_count, single_count, uniq_max, split_ribo, split_targ]]#, otu_count, chimera_count,  target_count, target_chimera]]
        count_table = count_table.append(summary)


count_table.columns = ['name','Read1','Read2','Merged_Reads','Filtered_Reads','Uniq_Reads','Single_Uniqs','Top_Uniq_Read_Count',"Split_16S_reads","Split_Target_Reads"]#,'Unsplit Uniqs','Unsplit Total Reads','OTU Count','Chimera Count', 'target clusters', 'target chimeras']


count_table.to_csv(WRK+'/processing_summary.txt', sep='\t')    
    


In [8]:
count_table.head(30)

,name,Read1,Read2,Merged_Reads,Filtered_Reads,Uniq_Reads,Single_Uniqs,Top_Uniq_Read_Count,Split_16S_reads,Split_Target_Reads
0,053-Single-B335-1_CTX-M-A,116817.0,116817.0,114381.0,108935,13866,11146,2507,13847.0,13847.0
0,054-Single-B335-2_CTX-M-A,51.0,51.0,11.0,11,11,11,1,9.0,9.0
0,055-Single-B335-3_CTX-M-A,109056.0,109056.0,106247.0,100791,10945,8470,3834,10928.0,10928.0
0,056-Single-NEG_CTX-M-A,25.0,25.0,0.0,3,3,3,1,0.0,0.0
0,057-Single-B335-1_TEM-A,79321.0,79321.0,76290.0,68914,5238,3528,15677,5221.0,5221.0
0,058-Single-B335-2_TEM-A,69813.0,69813.0,66651.0,59880,5093,3548,11285,5078.0,5078.0
0,059-Single-B335-3_TEM-A,72539.0,72539.0,69575.0,62817,6231,4523,10300,6214.0,6214.0
0,060-Single-NEG_TEM-A,90.0,90.0,68.0,60,12,7,14,12.0,12.0
0,061-Single-B335-1_CTX-M-B,116555.0,116555.0,114415.0,110175,12024,9405,3108,12002.0,12002.0
0,062-Single-B335-2_CTX-M-B,98936.0,98936.0,96845.0,92936,9647,7344,4194,9628.0,9628.0


In [7]:
 
    
    
        #COUNT UNSPLITTABLE READS
        try:
            with open(WRK+'/s5_split_reads/split_reads/unused_reads/unusedCounts.txt') as file:
                for  i, line in enumerate(file):
                    name=line.split("\t")[1]
                    if name==read:
                        k=line.split("\t")[2].strip().strip("[").strip("]")
                        unused_split=k.split(",")[0]
                        unused_total=k.split(",")[1]

        except IOError as e:
            print ("Unable to open "+read+" split read file") as file:
                for line in (file):
                    if 'Filtered reads (' in line:
                        line=line.split(' ')
                        for i in line:
                            if i.isdigit():
                                filter_count = i

   


        #COUNT OTUS
        try:
            with open(WRK+'/cluster_uniques_S6/16S_clusters/stats/'+read+'.txt') as file:
                if not "Fatal error" in open(WRK+'/cluster_uniques_S6/16S_clusters/stats/'+read+'.txt'):
                    for line in file:
                        if "100.0% " in line:
                            this_line = line.strip().split(' ')
                            print(this_line)
                            otu_count = this_line[6]
                            chimera_count = this_line[8]

        except IOError as e:
            print ("Unable to open "+read+" otu stats file")
             
        #COUNT target clusters
        try:
            with open(WRK+'/cluster_uniques_S6/target_clusters/stats/'+read+'.txt') as file:
                if not "Fatal error" in open(WRK+'/cluster_uniques_S6/target_clusters/stats/'+read+'.txt'):
                    for line in file:
                        if "100.0% " in line:
                            this_line = line.strip().split(' ')
                            print(this_line)
                            target_count = this_line[6]
                            target_chimera = this_line[8]

        except IOError as e:
            print ("Unable to open "+read+" otu stats file")        

        
        summary = [[read, R1_count, R2_count, merge_count, filter_count, uniq_count, single_count, uniq_max, unused_split, unused_total, otu_count, chimera_count,  target_count, target_chimera]]
        count_table = count_table.append(summary)


count_table.columns = ['name','Read1','Read2','Merged Reads','Filtered Reads','Uniq Reads','Single Uniqs','Top Uniq Read Count','Unsplit Uniqs','Unsplit Total Reads','OTU Count','Chimera Count', 'target clusters', 'target chimeras']


count_table.to_csv(WRK+'/processing_summary.txt', sep='\t')

['00:01', '47Mb', '', '', '', '100.0%', '19', 'OTUs,', '89', 'chimeras']
['00:00', '47Mb', '', '', '', '100.0%', '179', 'OTUs,', '0', 'chimeras']
['00:00', '43Mb', '', '', '', '100.0%', '1', 'OTUs,', '0', 'chimeras']
['00:00', '43Mb', '', '', '', '100.0%', '2', 'OTUs,', '0', 'chimeras']
['00:00', '47Mb', '', '', '', '100.0%', '11', 'OTUs,', '12', 'chimeras']
['00:00', '47Mb', '', '', '', '100.0%', '163', 'OTUs,', '0', 'chimeras']
['00:00', '43Mb', '', '', '', '100.0%', '1', 'OTUs,', '0', 'chimeras']
['00:00', '43Mb', '', '', '', '100.0%', '1', 'OTUs,', '0', 'chimeras']
['00:01', '47Mb', '', '', '', '100.0%', '28', 'OTUs,', '71', 'chimeras']
['00:00', '47Mb', '', '', '', '100.0%', '45', 'OTUs,', '0', 'chimeras']
['00:00', '47Mb', '', '', '', '100.0%', '8', 'OTUs,', '29', 'chimeras']
['00:00', '46Mb', '', '', '', '100.0%', '38', 'OTUs,', '0', 'chimeras']
['00:00', '47Mb', '', '', '', '100.0%', '11', 'OTUs,', '59', 'chimeras']
['00:00', '46Mb', '', '', '', '100.0%', '36', 'OTUs,', '1', 'c

In [14]:
try:
    with open(WRK+'/split_reads_S5/split_reads/unused_reads/unusedCounts.txt') as file:
        for  i, line in enumerate(file):
            name=line.split("\t")[1]
            if name==read:
                this=line.split("\t")[2]
                print(this)
            
except IOError as e:
    print ("Unable to open "+read+" split read file")      

            

70



In [24]:
this=line.split("\t")[2].strip().strip("[").strip("]")
this.split(",")[0]

'41'

In [142]:
            unused_split +=1
            if i>1:
                split_line = line.split('\t')
                split_line = split_line[1]
                split_line = split_line.split(';')[1].lstrip('size=')
                unused_total =unused_total+int(split_line)      



0

In [6]:
#count_table.to_csv(WRK+'/processing_summary.txt', sep='\t')
count_table.head(30)

,name,Read1,Read2,Merged Reads,Filtered Reads,Uniq Reads,Single Uniqs,Top Uniq Read Count,Unsplit Uniqs,Unsplit Total Reads,OTU Count,Chimera Count,target clusters,target chimeras
0,053-Single-B335-1_CTX-M-A,116817.0,116817.0,113905.0,112501,16367,13466,2530,801,1056,0,0,0,0
0,054-Single-B335-2_CTX-M-A,51.0,51.0,14.0,13,13,13,1,3,3,0,0,0,0
0,055-Single-B335-3_CTX-M-A,109056.0,109056.0,105680.0,104190,13438,10811,3855,543,948,0,0,0,0
0,056-Single-NEG_CTX-M-A,25.0,25.0,6.0,3,3,3,1,0,0,0,0,0,0
0,057-Single-B335-1_TEM-A,79321.0,79321.0,77444.0,72546,6807,4890,16017,503,1313,0,0,0,0
0,058-Single-B335-2_TEM-A,69813.0,69813.0,67770.0,63241,6707,4941,11515,428,1105,0,0,0,0
0,059-Single-B335-3_TEM-A,72539.0,72539.0,70764.0,66347,8125,6235,10470,490,1214,0,0,0,0
0,060-Single-NEG_TEM-A,90.0,90.0,71.0,64,15,10,15,0,0,0,0,0,0
0,061-Single-B335-1_CTX-M-B,116555.0,116555.0,114055.0,113034,13981,11200,3130,707,949,0,0,0,0
0,062-Single-B335-2_CTX-M-B,98936.0,98936.0,96510.0,95517,11473,9052,4225,560,930,0,0,0,0
